<a href="https://colab.research.google.com/github/florinOK/amr/blob/colab/sinkhorn-mnist/sinkhorn-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LEARNING LATENT PERMUTATIONS WITH GUMBEL-SINKHORN NETWORKS](https://arxiv.org/pdf/1802.08665.pdf)

---
https://github.com/google/gumbel_sinkhorn  
https://github.com/bbrattoli/JigsawPuzzlePytorch  

Задача отсортировать n чисел из отрезка [0, 1].

Каждому элементу последовательности (числу) xi поставили в соответствие вектор h. Размерность вектора равна длине последовательности, таким образом образуется квадратная матрица из векторов последовательности. К этой матрице применяется Sinkhorn оператор, в результате получается матрица перестановки для сортировки исходной последовательности.




In [1]:
! pip install tensorflow==1.15.2

     |████████████████████████████████| 110.5MB 62kB/s 
     |████████████████████████████████| 3.8MB 45.1MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 512kB 42.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=f4468908337be9d74ab097efd198426a0a8307b63f4a35abc88e7ee35dac5a6c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [2]:
import numpy as np
import tensorflow as tf

# optimizer.py

In [4]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


"""Library with optimization definitions and functions."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


def set_optimizer(optimizer, lr, opt_eps=1.0, opt_momentum=0.9, rms_decay=0.9,
                  adam_beta1=0.9, adam_beta2=0.999):
  """Sets optimizer optimizer op.

  Args:
   optimizer: A string (sgd, momentum, adagrad, adam, rmsprop).
   lr: learning rate, a float.
   opt_eps: Optimizer epsilon (for ADAM and RMSprop).
   opt_momentum: Optimizer momentum. Common for Momentum and RMSProp.
   rms_decay: RMSProp decay parameter.
   adam_beta1: beta_1 parameter for ADAM.
   adam_beta2: beta_2 parameter for ADAM.
  Returns:
    opt, the optimizer op.

  """

  if optimizer == "sgd":
    opt = tf.train.GradientDescentOptimizer(lr)
  elif optimizer == "momentum":
    opt = tf.train.MomentumOptimizer(lr, opt_momentum)
  elif optimizer == "adagrad":
    opt = tf.train.AdagradOptimizer(lr)
  elif optimizer == "adam":
    opt = tf.train.AdamOptimizer(lr, beta1=adam_beta1, beta2=adam_beta2,
                                 epsilon=opt_eps)
  elif optimizer == "rmsprop":
    opt = tf.train.RMSPropOptimizer(lr, rms_decay, opt_momentum, opt_eps)
  return opt


# sinkhorn_ops.py

In [6]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""A tensorflow lib of ops with permutations, and sinkhorn balancing.

A tensorflow library of operations and sampling with permutations
and their approximation with doubly-stochastic matrices, through Sinkhorn
balancing

"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.stats import kendalltau
import tensorflow as tf


def sample_gumbel(shape, eps=1e-20):
  """Samples arbitrary-shaped standard gumbel variables.

  Args:
    shape: list of integers
    eps: float, for numerical stability
  Returns:
    A sample of standard Gumbel random variables
  """

  u = tf.random_uniform(shape, minval=0, maxval=1, dtype=tf.float32)
  return -tf.log(-tf.log(u + eps) + eps)


def matching(matrix_batch):
  """Solves a matching problem for a batch of matrices.

  This is a wrapper for the scipy.optimize.linear_sum_assignment function. It
  solves the optimization problem max_P sum_i,j M_i,j P_i,j with P a
  permutation matrix. Notice the negative sign; the reason, the original
  function solves a minimization problem

  Args:
    matrix_batch: A 3D tensor (a batch of matrices) with
      shape = [batch_size, N, N]. If 2D, the input is reshaped to 3D with
      batch_size = 1.

  Returns:
    listperms, a 2D integer tensor of permutations with shape [batch_size, N]
      so that listperms[n, :] is the permutation of range(N) that solves the
      problem  max_P sum_i,j M_i,j P_i,j with M = matrix_batch[n, :, :].
  """

  def hungarian(x):
    if x.ndim == 2:
      x = np.reshape(x, [1, x.shape[0], x.shape[1]])
    sol = np.zeros((x.shape[0], x.shape[1]), dtype=np.int32)
    for i in range(x.shape[0]):
      sol[i, :] = linear_sum_assignment(-x[i, :])[1].astype(np.int32)
    return sol

  listperms = tf.py_func(hungarian, [matrix_batch], tf.int32)
  return listperms


def kendall_tau(batch_perm1, batch_perm2):
  """Wraps scipy.stats kendalltau function.

  Args:
    batch_perm1: A 2D tensor (a batch of matrices) with
      shape = [batch_size, N]
    batch_perm2: same as batch_perm1

  Returns:
    A list of Kendall distances between each of the elements of the batch.
  """

  def kendalltau_batch(x, y):

    if x.ndim == 1:
      x = np.reshape(x, [1, x.shape[0]])
    if y.ndim == 1:
      y = np.reshape(y, [1, y.shape[0]])
    kendall = np.zeros((x.shape[0], 1), dtype=np.float32)
    for i in range(x.shape[0]):
      kendall[i, :] = kendalltau(x[i, :], y[i, :])[0]
    return kendall

  listkendall = tf.py_func(kendalltau_batch, [batch_perm1, batch_perm2], tf.float32)
  return listkendall


def sinkhorn(log_alpha, n_iters=20):
  """Performs incomplete Sinkhorn normalization to log_alpha.

  By a theorem by Sinkhorn and Knopp [1], a sufficiently well-behaved  matrix
  with positive entries can be turned into a doubly-stochastic matrix
  (i.e. its rows and columns add up to one) via the succesive row and column
  normalization.
  -To ensure positivity, the effective input to sinkhorn has to be
  exp(log_alpha) (elementwise).
  -However, for stability, sinkhorn works in the log-space. It is only at
   return time that entries are exponentiated.

  [1] Sinkhorn, Richard and Knopp, Paul.
  Concerning nonnegative matrices and doubly stochastic
  matrices. Pacific Journal of Mathematics, 1967

  Args:
    log_alpha: 2D tensor (a matrix of shape [N, N])
      or 3D tensor (a batch of matrices of shape = [batch_size, N, N])
    n_iters: number of sinkhorn iterations (in practice, as little as 20
      iterations are needed to achieve decent convergence for N~100)

  Returns:
    A 3D tensor of close-to-doubly-stochastic matrices (2D tensors are
      converted to 3D tensors with batch_size equals to 1)
  """
  n = tf.shape(log_alpha)[1]
  log_alpha = tf.reshape(log_alpha, [-1, n, n])

  for _ in range(n_iters):
    log_alpha -= tf.reshape(tf.reduce_logsumexp(log_alpha, axis=2), [-1, n, 1])
    log_alpha -= tf.reshape(tf.reduce_logsumexp(log_alpha, axis=1), [-1, 1, n])
  return tf.exp(log_alpha)


def gumbel_sinkhorn(log_alpha, temp=1.0, n_samples=1, noise_factor=1.0, n_iters=20, squeeze=True):
  """Random doubly-stochastic matrices via gumbel noise.

  In the zero-temperature limit sinkhorn(log_alpha/temp) approaches
  a permutation matrix. Therefore, for low temperatures this method can be
  seen as an approximate sampling of permutation matrices, where the
  distribution is parameterized by the matrix log_alpha

  The deterministic case (noise_factor=0) is also interesting: it can be
  shown that lim t->0 sinkhorn(log_alpha/t) = M, where M is a
  permutation matrix, the solution of the
  matching problem M=arg max_M sum_i,j log_alpha_i,j M_i,j.
  Therefore, the deterministic limit case of gumbel_sinkhorn can be seen
  as approximate solving of a matching problem, otherwise solved via the
  Hungarian algorithm.

  Warning: the convergence holds true in the limit case n_iters = infty.
  Unfortunately, in practice n_iter is finite which can lead to numerical
  instabilities, mostly if temp is very low. Those manifest as
  pseudo-convergence or some row-columns to fractional entries (e.g.
  a row having two entries with 0.5, instead of a single 1.0)
  To minimize those effects, try increasing n_iter for decreased temp.
  On the other hand, too-low temperature usually lead to high-variance in
  gradients, so better not choose too low temperatures.

  Args:
    log_alpha: 2D tensor (a matrix of shape [N, N])
      or 3D tensor (a batch of matrices of shape = [batch_size, N, N])
    temp: temperature parameter, a float.
    n_samples: number of samples
    noise_factor: scaling factor for the gumbel samples. Mostly to explore
      different degrees of randomness (and the absence of randomness, with
      noise_factor=0)
    n_iters: number of sinkhorn iterations. Should be chosen carefully, in
      inverse corresponde with temp to avoid numerical stabilities.
    squeeze: a boolean, if True and there is a single sample, the output will
      remain being a 3D tensor.

  Returns:
    sink: a 4D tensor of [batch_size, n_samples, N, N] i.e.
      batch_size *n_samples doubly-stochastic matrices. If n_samples = 1 and
      squeeze = True then the output is 3D.
    log_alpha_w_noise: a 4D tensor of [batch_size, n_samples, N, N] of
      noisy samples of log_alpha, divided by the temperature parameter. If
      n_samples = 1 then the output is 3D.
  """
  n = tf.shape(log_alpha)[1]
  log_alpha = tf.reshape(log_alpha, [-1, n, n])
  batch_size = tf.shape(log_alpha)[0]
  log_alpha_w_noise = tf.tile(log_alpha, [n_samples, 1, 1])
  if noise_factor == 0:
    noise = 0.0
  else:
    noise = sample_gumbel([n_samples*batch_size, n, n])*noise_factor
  log_alpha_w_noise += noise
  log_alpha_w_noise /= temp
  sink = sinkhorn(log_alpha_w_noise, n_iters)
  if n_samples > 1 or squeeze is False:
    sink = tf.reshape(sink, [n_samples, batch_size, n, n])
    sink = tf.transpose(sink, [1, 0, 2, 3])
    log_alpha_w_noise = tf.reshape(
        log_alpha_w_noise, [n_samples, batch_size, n, n])
    log_alpha_w_noise = tf.transpose(log_alpha_w_noise, [1, 0, 2, 3])
  return sink, log_alpha_w_noise


def sample_uniform_and_order(n_lists, n_numbers, prob_inc):
  """Samples uniform random numbers, and sort them.

  Returns a 2-D tensor of n_lists lists of n_numbers sorted numbers in the [0,1]
  interval, each of them having n_numbers elements.
  Lists are increasing with probability prob_inc.
  It does so by first sampling uniform random numbers, and then sorting them.
  Therefore, sorted numbers follow the distribution of the order statistics of
  a uniform distribution.
  It also returns the random numbers and the lists of permutations p such
  p(sorted) = random.
  Notice that if one ones to build sorted numbers in different intervals, one
  might just want to re-scaled this canonical form.

  Args:
    n_lists: An int,the number of lists to be sorted.
    n_numbers: An int, the number of elements in the permutation.
    prob_inc: A float, the probability that an list of numbers will be sorted in
    increasing order.

  Returns:
   ordered: a 2-D float tensor with shape = [n_list, n_numbers] of sorted lists
     of numbers.
   random: a 2-D float tensor with shape = [n_list, n_numbers] of uniform random
     numbers.
   permutations: a 2-D int tensor with shape = [n_list, n_numbers], row i
     satisfies ordered[i, permutations[i]) = random[i,:].

  """

  bern = tf.contrib.distributions.Bernoulli(
      probs=np.ones((n_lists, 1)) * prob_inc).sample()
  sign = -1*tf.cast(tf.multiply(bern, 2) -1, dtype=tf.float32)
  random = tf.random_uniform(shape=[n_lists, n_numbers], dtype=tf.float32)
  random_with_sign = tf.multiply(random, sign)
  ordered, permutations = tf.nn.top_k(random_with_sign, k=n_numbers)
  ordered = tf.multiply(ordered, sign)
  return ordered, random, permutations


def sample_permutations(n_permutations, n_objects):
  """Samples a batch permutations from the uniform distribution.

  Returns a sample of n_permutations permutations of n_objects indices.
  Permutations are assumed to be represented as lists of integers
  (see 'listperm2matperm' and 'matperm2listperm' for conversion to alternative
  matricial representation). It does so by sampling from a continuous
  distribution and then ranking the elements. By symmetry, the resulting
  distribution over permutations must be uniform.

  Args:
    n_permutations: An int, the number of permutations to sample.
    n_objects: An int, the number of elements in the permutation.
      the embedding sources.

  Returns:
    A 2D integer tensor with shape [n_permutations, n_objects], where each
      row is a permutation of range(n_objects)

  """

  random_pre_perm = tf.random_normal(shape=[n_permutations, n_objects])
  _, permutations = tf.nn.top_k(random_pre_perm, k=n_objects)
  return permutations


def permute_batch_split(batch_split, permutations):
  """Scrambles a batch of objects according to permutations.

  It takes a 3D tensor [batch_size, n_objects, object_size]
  and permutes items in axis=1 according to the 2D integer tensor
  permutations, (with shape [batch_size, n_objects]) a list of permutations
  expressed as lists. For many dimensional-objects (e.g. images), objects have
  to be flattened so they will respect the 3D format, i.e. tf.reshape(
  batch_split, [batch_size, n_objects, -1])

  Args:
    batch_split: 3D tensor with shape = [batch_size, n_objects, object_size] of
      splitted objects
    permutations: a 2D integer tensor with shape = [batch_size, n_objects] of
      permutations, so that permutations[n] is a permutation of range(n_objects)

  Returns:
    A 3D tensor perm_batch_split with the same shape as batch_split,
      so that perm_batch_split[n, j,:] = batch_split[n, perm[n,j],:]

  """

  batch_split_size = tf.shape(batch_split, out_type=permutations.dtype)[0]
  n_objects = tf.shape(batch_split)[1]

  ind_permutations = tf.reshape(permutations, [-1, 1])

  ind_batch = tf.reshape(tf.tile(tf.reshape(tf.range(batch_split_size),
                                            [-1, 1]),
                                 [1, n_objects]),
                         [-1, 1])

  ind_batch_and_permutation = tf.concat((ind_batch, ind_permutations), axis=1)

  batch_split = tf.reshape(tf.gather_nd(batch_split, ind_batch_and_permutation),
                           [batch_split_size, n_objects, -1])

  return batch_split


def listperm2matperm(listperm):
  """Converts a batch of permutations to its matricial form.

  Args:
    listperm: 2D tensor of permutations of shape [batch_size, n_objects] so that
      listperm[n] is a permutation of range(n_objects).

  Returns:
    a 3D tensor of permutations matperm of
      shape = [batch_size, n_objects, n_objects] so that matperm[n, :, :] is a
      permutation of the identity matrix, with matperm[n, i, listperm[n,i]] = 1
  """
  n_objects = tf.shape(listperm)[1]
  return tf.one_hot(listperm, n_objects)


def matperm2listperm(matperm, dtype=tf.int32):
  """Converts a batch of permutations to its enumeration (list) form.

  Args:
    matperm: a 3D tensor of permutations of
      shape = [batch_size, n_objects, n_objects] so that matperm[n, :, :] is a
      permutation of the identity matrix. If the input is 2D, it is reshaped
      to 3D with batch_size = 1.
    dtype: output_type (tf.int32, tf.int64)

  Returns:
    A 2D tensor of permutations listperm, where listperm[n,i]
    is the index of the only non-zero entry in matperm[n, i, :]
  """
  matperm = tf.reshape(matperm, [-1,
                                 tf.shape(matperm)[1], tf.shape(matperm)[1]])
  batch_size = tf.shape(matperm)[0]
  n_objects = tf.shape(matperm)[1]

  return tf.reshape(tf.argmax(matperm, axis=2, output_type=dtype),
                    [batch_size, n_objects])


def invert_listperm(listperm):
  """Inverts a batch of permutations.

  Args:
    listperm: a 2D integer tensor of permutations listperm of
      shape = [batch_size, n_objects] so that listperm[n] is a permutation of
      range(n_objects)
  Returns:
    A 2D tensor of permutations listperm, where listperm[n,i]
    is the index of the only non-zero entry in matperm[n, i, :]
  """
  return matperm2listperm(tf.transpose(listperm2matperm(listperm), [0, 2, 1]))


# sorting_model.py

In [22]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


"""Model class for sorting numbers.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


class SortingModel(object):
  """Constructs the graph of tensors to learn to sort numbers."""

  def __init__(self, graph, hparams):
    self.graph = graph
    self.hparams = hparams
    self.batch_size = hparams.batch_size
    self.n_numbers = hparams.n_numbers
    self.samples_per_num = hparams.samples_per_num
    self.n_iter_sinkhorn = hparams.n_iter_sinkhorn
    self.noise_factor = hparams.noise_factor
    self.prob_inc = hparams.prob_inc
    self.optimizer = hparams.optimizer
    self.n_units = hparams.n_units

  def set_input(self):
    with self.graph.as_default():
      (self._ordered, self._random, self._hard_perms) = \
          sample_uniform_and_order(self.batch_size, self.n_numbers, self.prob_inc)
      # tiled variables, to compare to many permutations
      self._ordered_tiled = tf.tile(self._ordered, [self.samples_per_num, 1])
      self._random_tiled = tf.tile(self._random, [self.samples_per_num, 1])

  def build_network(self):
    """The most important part, where the neural network is built."""

    def _create_log_alpha(self):
      """Creates the variable log_alpha, through NN processing of input."""

      with tf.variable_scope("model_params"):
        # each number is processed with the same network, so data is reshaped
        # so that numbers occupy the 'batch' position.
        random_flattened = tf.reshape(self._random, [-1, 1])
        # net: output of the first neural network that connects numbers to a
        # 'latent' representation.
        net = dropout(fc(random_flattened, self.n_units), self.keep_prob)
        # now those latent representation is connected to rows of the matrix
        # log_alpha.
        processed = dropout(fc(net, self.n_numbers, activation_fn=None), self.keep_prob)

        # the matrix log_alpha is created by concatenation of the rows
        # corresponding to different numbers.
        return tf.reshape(processed, [-1, self.n_numbers, self.n_numbers])

    with self.graph.as_default():

      self.keep_prob = tf.constant(self.hparams.keep_prob, dtype=tf.float32)
      self.temperature = tf.constant(self.hparams.temperature, dtype=tf.float32)
      self._global_step = tf.Variable(0, trainable=False)
      fc = tf.contrib.layers.fully_connected
      dropout = tf.contrib.layers.dropout

      self._log_alpha = _create_log_alpha(self)
      # Now, we sample using gumbel_sinkhorn from the
      # constructed matrix log_alpha.
      (self._soft_perms_inf, self._log_alpha_w_noise) = gumbel_sinkhorn(
          self._log_alpha, self.temperature, self.samples_per_num,
          self.noise_factor, self.n_iter_sinkhorn, squeeze=False
      )

  def build_initializer(self):
    with self.graph.as_default():
      tf.initialize_all_variables()

  def build_l2s_loss(self):
    """Builds loss tensor with soft permutations, for training."""
    with self.graph.as_default():
      inv_soft_perms = tf.transpose(self._soft_perms_inf, [0, 1, 3, 2])
      inv_soft_perms_flat = tf.reshape(tf.transpose(inv_soft_perms, [1, 0, 2, 3]), [-1, self.n_numbers, self.n_numbers])
      ordered_tiled = tf.reshape(self._ordered_tiled, [-1, self.n_numbers, 1])
      random_tiled = tf.reshape(self._random_tiled, [-1, self.n_numbers, 1])
      # squared l2 loss
      self._l2s_diff = tf.reduce_mean(tf.square(ordered_tiled - tf.matmul(inv_soft_perms_flat, random_tiled)))

  def build_hard_losses(self):
    """Losses based on hard reconstruction. Only for evaluation.

    Doubly stochastic matrices are rounded with
    the matching function.
    """

    log_alpha_w_noise = tf.transpose(self._log_alpha_w_noise, [1, 0, 2, 3])
    log_alpha_w_noise_flat = tf.reshape(log_alpha_w_noise, [-1, self.n_numbers, self.n_numbers])

    hard_perms_inf = matching(log_alpha_w_noise_flat)
    inverse_hard_perms_inf = invert_listperm(hard_perms_inf)
    hard_perms_tiled = tf.tile(self._hard_perms, [self.samples_per_num, 1])

    # The 3D output of permute_batch_split must be squeezed
    _permute_batch_split = permute_batch_split(self._random_tiled, inverse_hard_perms_inf)
    self._ordered_inf_tiled = tf.reshape(_permute_batch_split, [-1, self.n_numbers])

    self._l1_diff = tf.reduce_mean(tf.abs(self._ordered_tiled - self._ordered_inf_tiled))
    self._l2sh_diff = tf.reduce_mean(tf.square(self._ordered_tiled - self._ordered_inf_tiled))

    diff_perms = tf.cast(tf.abs(hard_perms_tiled - inverse_hard_perms_inf), tf.float32)
    self._prop_wrong = -tf.reduce_mean(tf.sign(-diff_perms))
    self._prop_any_wrong = -tf.reduce_mean(tf.sign(-tf.reduce_sum(diff_perms, 1)))
    self._kendall_tau = tf.reduce_mean(kendall_tau(hard_perms_tiled, inverse_hard_perms_inf))

  def build_optimizer(self):
    with self.graph.as_default():
      opt = set_optimizer(self.hparams.optimizer, self.hparams.lr, opt_eps=1e-8)
      self._train_op = tf.contrib.training.create_train_op(
          self._l2s_diff, opt, global_step=self._global_step)

  def build_train_ops(self):
    with self.graph.as_default():
      self._vars = tf.trainable_variables()
      self._train_op = tf.train.AdamOptimizer(self.hparams.lr).minimize(self._l2s_diff,
                                                                        var_list=self._vars,
                                                                        global_step=self._global_step)

  def add_summaries_train(self):
    """Adds necessary summaries which will be computed during training."""
    with tf.name_scope("Training"):
      with self.graph.as_default():
        tf.summary.scalar("Total_l2_squared_loss", self._l2s_diff)

  def add_summaries_eval(self):
    """Adds necessary summaries which will be computed during evaluation."""
    with tf.name_scope("Evaluation"):
      with self.graph.as_default():
        tf.summary.scalar("L1_diff", self._l1_diff)
        tf.summary.scalar("L2_squared_diff", self._l2sh_diff)
        tf.summary.scalar("Proportion_wrong", self._prop_wrong)
        tf.summary.scalar("Proportion_where_any_is_wrong", self._prop_any_wrong)
        tf.summary.scalar("Kendall's_tau", self._kendall_tau)

  def get_eval_measures(self):
    """Getter method for evaluation measures."""
    return (self._l1_diff, self._l2sh_diff, self._kendall_tau, self._prop_wrong, self._prop_any_wrong)

  @property
  def train_op(self):
    return self._train_op

  @property
  def ordered_inf(self):
    return tf.transpose(tf.reshape(self._ordered_inf_tiled, [self.samples_per_num, -1, self.n_numbers]), [1, 0, 2])

  @property
  def ordered(self):
    return tf.transpose(tf.reshape(self._ordered_tiled, [self.samples_per_num, -1, self.n_numbers]), [1, 0, 2])

  @property
  def random(self):
    return tf.transpose(tf.reshape(self._random_tiled, [self.samples_per_num, -1, self.n_numbers]), [1, 0, 2])

  @property
  def global_step(self):
    return self._global_step


# sorting_train.py

In [17]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Trains a model that sorts numbers, keeping loss summaries in tensorboard.

The flag hparam has to be passed as a string of comma separated statements of
the form hparam=value, where the hparam's are any of the listed in the
dictionary DEFAULT_HPARAMS.
See the README.md file for further compilation and running instructions.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


flags = tf.app.flags
gfile = tf.gfile

DEFAULT_HPARAMS = tf.contrib.training.HParams(hparams='',
                                              num_iters=500,
                                              save_summaries_secs=30,
                                              save_interval_secs=30,
                                              exp_log_dir='./tmp_sorting/',
                                              max_to_keep=1,
                                              #
                                              n_numbers=50,
                                              lr=0.1,
                                              temperature=1.0,
                                              batch_size=10,
                                              prob_inc=1.0,
                                              samples_per_num=5,
                                              n_iter_sinkhorn=10,
                                              n_units=32,
                                              noise_factor=1.0,
                                              optimizer='adam',
                                              keep_prob=1.)


def main(_):

  hparams = DEFAULT_HPARAMS

  if not gfile.Exists(hparams.exp_log_dir):
    gfile.MakeDirs(hparams.exp_log_dir)
  tf.reset_default_graph()
  g = tf.Graph()
  model = SortingModel(g, hparams)
  with g.as_default():
    model.set_input()
    model.build_network()
    model.build_l2s_loss()
    model.build_optimizer()
    model.add_summaries_train()

    with tf.Session():
      tf.contrib.slim.learning.train(
          train_op=model.train_op,
          logdir=hparams.exp_log_dir,
          global_step=model.global_step,
          saver=tf.train.Saver(max_to_keep=hparams.max_to_keep),
          number_of_steps=hparams.num_iters,
          save_summaries_secs=hparams.save_summaries_secs,
          save_interval_secs=hparams.save_interval_secs)


if __name__ == '__main__':
  tf.app.run(main)


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


W1007 10:11:43.495110 140316417697664 deprecation.py:323] From <ipython-input-6-5efadca2e524>:237: Bernoulli.__init__ (from tensorflow.python.ops.distributions.bernoulli) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


W1007 10:11:43.507371 140316417697664 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/distributions/bernoulli.py:97: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
Please use `layer.__call__` method instead.


...

I1007 10:12:05.366161 140316417697664 learning.py:785] Finished training! Saving model to disk.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


W1007 10:12:05.375056 140316417697664 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/summary/writer/writer.py:386: UserWarning: Attempting to use a closed FileWriter. The operation will be a noop unless the FileWriter is explicitly reopened.
  warnings.warn("Attempting to use a closed FileWriter. "


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# sorting_eval.py

In [24]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Evaluates a sorting model, adding scalar summaries to tensorboard.

It also outputs evaluation metrics as numpy scalars.
The flag hparam has to be passed as a string of comma separated statements of
the form hparam=value, where the hparam's are any of the listed in the
dictionary DEFAULT_HPARAMS.
See the README.md file for further compilation and running instructions.

"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import os
import time

import tensorflow as tf


DEFAULT_HPARAMS = tf.contrib.training.HParams(hparams='',
                                              batch_transform_type=None,
                                              evaluate_all=True,
                                              exp_log_dir='./tmp_sorting/',
                                              eval_once=True,
                                              sec_sleep=30,
                                              secs_run_for=10000,
                                              #
                                              n_numbers=50,
                                              lr=0.1,
                                              temperature=1.0,
                                              batch_size=50,
                                              prob_inc=1.0,
                                              samples_per_num=5,
                                              n_iter_sinkhorn=10,
                                              n_units=32,
                                              noise_factor=0.0,
                                              optimizer="adam",
                                              keep_prob=1.)


def log(s):
  tf.logging.info(s)
  print(s)


def wait_for_new_checkpoint(saver, sess, logdir, global_step,
                            last_step_evaluated, sleep_secs):
  while True:
    if restore_checkpoint_if_exists(saver, sess, logdir):
      step = sess.run(global_step)
      if step <= last_step_evaluated:
        log("Found old checkpoint, sleeping %ds" % sleep_secs)
        time.sleep(sleep_secs)
      else:
        return step
    else:
      log("Checkpoint not found in %s,"
          "sleeping for %ds" % (logdir, sleep_secs))
      time.sleep(sleep_secs)


def restore_checkpoint_if_exists(saver, sess, logdir):
  ckpt = tf.train.get_checkpoint_state(logdir)
  if ckpt:
    log("Restoring checking point from %s" % ckpt.model_checkpoint_path)
    ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
    full_checkpoint_path = os.path.join(logdir, ckpt_name)
    saver.restore(sess, full_checkpoint_path)
    return True
  return False


# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

def main(_):
  time_start = time.time()
  hparams = DEFAULT_HPARAMS

  if not gfile.Exists(hparams.exp_log_dir):
    gfile.MakeDirs(hparams.exp_log_dir)

  g = tf.Graph()
  model = SortingModel(g, hparams)

  with model.graph.as_default():
    model.set_input()
    model.build_network()
    model.build_hard_losses()
    model.add_summaries_eval()
    summaries_eval = tf.summary.merge_all()
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(hparams.exp_log_dir, model.graph)
    last_step_evaluated = -1
    with tf.Session() as session:
      while time.time() - time_start < hparams.secs_run_for:
        wait_for_new_checkpoint(saver, session, hparams.exp_log_dir, model.global_step, last_step_evaluated, hparams.sec_sleep)
        (summaries, step, eval_measures) = session.run([summaries_eval, model.global_step, model.get_eval_measures()])
        (l1_diff, l2sh_diff, kendall_tau, prop_wrong, prop_any_wrong) = eval_measures
        log(f"Frequency of mistakes was {prop_wrong:.4f}")
        log(f"Frequency of series with at least an error was {prop_any_wrong:.4f}")
        log(f"Kendall's tau was %s" % kendall_tau)
        log(f"Mean L2 squared difference between true and inferred series was {l2sh_diff:.4f}")
        log(f"Mean L1 difference between true and inferred series was {l1_diff:.4f}")
        writer.add_summary(summaries, global_step=step)
        last_step_evaluated = step
        if hparams.eval_once is True:
          break

if __name__ == "__main__":
  tf.app.run()


INFO:tensorflow:Summary name Kendall's_tau is illegal; using Kendall_s_tau instead.


I1007 10:20:43.798891 140316417697664 summary_op_util.py:66] Summary name Kendall's_tau is illegal; using Kendall_s_tau instead.


INFO:tensorflow:Restoring checking point from ./tmp_sorting/model.ckpt-500


I1007 10:20:43.881588 140316417697664 <ipython-input-24-8ac1663a03f6>:58] Restoring checking point from ./tmp_sorting/model.ckpt-500


Restoring checking point from ./tmp_sorting/model.ckpt-500
INFO:tensorflow:Restoring parameters from ./tmp_sorting/model.ckpt-500


...

Mean L2 squared difference between true and inferred series was 0.0000
INFO:tensorflow:Mean L1 difference between true and inferred series was 0.0005


I1007 10:20:44.096320 140316417697664 <ipython-input-24-8ac1663a03f6>:58] Mean L1 difference between true and inferred series was 0.0005


Mean L1 difference between true and inferred series was 0.0005


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# main

In [85]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import os
import time

import tensorflow as tf


DEFAULT_HPARAMS = tf.contrib.training.HParams(hparams='',
                                              batch_transform_type=None,
                                              evaluate_all=True,
                                              exp_log_dir='./tmp_sorting/',
                                              eval_once=True,
                                              sec_sleep=30,
                                              secs_run_for=10000,
                                              #
                                              n_numbers=50,
                                              lr=0.1,
                                              temperature=1.0,
                                              batch_size=50,
                                              prob_inc=1.0,
                                              samples_per_num=5,
                                              n_iter_sinkhorn=10,
                                              n_units=32,
                                              noise_factor=0.0,
                                              optimizer="adam",
                                              keep_prob=1.)


def main():
  time_start = time.time()
  hparams = DEFAULT_HPARAMS
  
  if not gfile.Exists(hparams.exp_log_dir):
    gfile.MakeDirs(hparams.exp_log_dir)

  g = tf.Graph()
  model = SortingModel(g, hparams)

  with model.graph.as_default():
    model.set_input()
    model.build_network()
    model.build_hard_losses()


    log_alpha_w_noise = tf.transpose(model._log_alpha_w_noise, [1, 0, 2, 3])
    log_alpha_w_noise_flat = tf.reshape(log_alpha_w_noise, [-1, model.n_numbers, model.n_numbers])
    hard_perms_inf = matching(log_alpha_w_noise_flat)
    inverse_hard_perms_inf = invert_listperm(hard_perms_inf)
    # hard_perms_inf = tf.reshape(hard_perms_inf, [model.samples_per_num, model.batch_size, model.n_numbers])
    # hard_perms_inf = tf.transpose(hard_perms_inf, [1, 0, 2])


    hard_perms_tiled = tf.tile(model._hard_perms, [model.samples_per_num, 1])
    diff_perms = tf.cast(tf.abs(hard_perms_tiled - inverse_hard_perms_inf), tf.float32)


    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(hparams.exp_log_dir, model.graph)
    last_step_evaluated = -1
    with tf.Session() as session:
        wait_for_new_checkpoint(saver, session, hparams.exp_log_dir, model.global_step, last_step_evaluated, hparams.sec_sleep)

        ops = {
            'model._ordered': model._ordered,
            'model._random': model._random,
            'model._hard_perms': model._hard_perms,

            'model._ordered_tiled': model._ordered_tiled,
            'model._random_tiled': model._random_tiled,
            'model._hard_perms_tiled': hard_perms_tiled,

            'model._hard_perms_inf': hard_perms_inf,
            'model._inverse_hard_perms_inf': inverse_hard_perms_inf,
            
            'model._diff_perms': diff_perms,

            'model._soft_perms_inf': model._soft_perms_inf,
            'model._log_alpha_w_noise': model._log_alpha_w_noise
        }
        results = session.run(ops)

        for op, result in results.items():
            print(op, result.shape)

    return results


results = main()

INFO:tensorflow:Restoring checking point from ./tmp_sorting/model.ckpt-500


I1007 11:18:31.964698 140316417697664 <ipython-input-24-8ac1663a03f6>:58] Restoring checking point from ./tmp_sorting/model.ckpt-500


Restoring checking point from ./tmp_sorting/model.ckpt-500
INFO:tensorflow:Restoring parameters from ./tmp_sorting/model.ckpt-500


I1007 11:18:31.967840 140316417697664 saver.py:1284] Restoring parameters from ./tmp_sorting/model.ckpt-500


model._ordered (50, 50)
model._random (50, 50)
model._hard_perms (50, 50)
model._ordered_tiled (250, 50)
model._random_tiled (250, 50)
model._hard_perms_tiled (250, 50)
model._hard_perms_inf (250, 50)
model._inverse_hard_perms_inf (250, 50)
model._diff_perms (250, 50)
model._soft_perms_inf (50, 5, 50, 50)
model._log_alpha_w_noise (50, 5, 50, 50)


In [105]:
results['model._random'][0]

array([0.09693146, 0.93467736, 0.8104842 , 0.8827152 , 0.26225126,
       0.97478354, 0.75638855, 0.06581938, 0.33848166, 0.16084397,
       0.5362569 , 0.2828995 , 0.800158  , 0.5055257 , 0.5884433 ,
       0.5605475 , 0.5194547 , 0.8333657 , 0.7619628 , 0.26023293,
       0.23527515, 0.38149345, 0.53956866, 0.9372901 , 0.98261154,
       0.04836261, 0.17472851, 0.20129049, 0.86930656, 0.05879653,
       0.12261772, 0.03646851, 0.2853694 , 0.34873128, 0.24351108,
       0.16725624, 0.49957192, 0.96747124, 0.20655656, 0.28190684,
       0.11749411, 0.51000047, 0.808496  , 0.45947385, 0.81732714,
       0.32300103, 0.27582836, 0.70436287, 0.02080083, 0.1342628 ],
      dtype=float32)

In [106]:
results['model._ordered'][0]

array([0.02080083, 0.03646851, 0.04836261, 0.05879653, 0.06581938,
       0.09693146, 0.11749411, 0.12261772, 0.1342628 , 0.16084397,
       0.16725624, 0.17472851, 0.20129049, 0.20655656, 0.23527515,
       0.24351108, 0.26023293, 0.26225126, 0.27582836, 0.28190684,
       0.2828995 , 0.2853694 , 0.32300103, 0.33848166, 0.34873128,
       0.38149345, 0.45947385, 0.49957192, 0.5055257 , 0.51000047,
       0.5194547 , 0.5362569 , 0.53956866, 0.5605475 , 0.5884433 ,
       0.70436287, 0.75638855, 0.7619628 , 0.800158  , 0.808496  ,
       0.8104842 , 0.81732714, 0.8333657 , 0.86930656, 0.8827152 ,
       0.93467736, 0.9372901 , 0.96747124, 0.97478354, 0.98261154],
      dtype=float32)

In [107]:
results['model._hard_perms'][0]

array([48, 31, 25, 29,  7,  0, 40, 30, 49,  9, 35, 26, 27, 38, 20, 34, 19,
        4, 46, 39, 11, 32, 45,  8, 33, 21, 43, 36, 13, 41, 16, 10, 22, 15,
       14, 47,  6, 18, 12, 42,  2, 44, 17, 28,  3,  1, 23, 37,  5, 24],
      dtype=int32)

In [108]:
# check random[perms] = ordered
results['model._random'][0][results['model._hard_perms'][0]] == results['model._ordered'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [113]:
results['model._hard_perms'][0] == results['model._hard_perms_tiled'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [114]:
# check random[perms] = ordered
results['model._random'][0][results['model._hard_perms_tiled'][0]] == results['model._ordered'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [115]:
results['model._inverse_hard_perms_inf'][0] == results['model._hard_perms_tiled'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [116]:
# check random[perms] = ordered
results['model._random'][0][results['model._inverse_hard_perms_inf'][0]] == results['model._ordered'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])